## Testing end-to-end

In [1]:
import json
import sys
import os
sys.path.append('/Users/gisaac/Documents/GitHub/river_flow_notebooks/')
with open("/Users/gisaac/Documents/GitHub/river_flow_notebooks/tests/test_data/test_meta.json") as f:
    data = json.load(f)

In [17]:
from scraping_functions import HydroScraper

In [18]:
from datetime import datetime
h = HydroScraper(datetime(2020, 1, 1), datetime(2020, 1, 1), "/Users/gisaac/Documents/GitHub/river_flow_notebooks/tests/test_data/test_meta.json")

Getting request from USGS
http://waterservices.usgs.gov/nwis/iv/?format=rdb,1.0&sites=01010000&startDT=2020-01-01&endDT=2020-01-01&parameterCd=00060,00065,00045&siteStatus=all
Request finished
Getting request from ASOS
https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=40B&data=tmpf&data=dwpf&data=relh&data=feel&data=sknt&data=sped&data=alti&data=mslp&data=drct&data=ice_accretion_1hr&data=p01m&data=vsby&data=gust&data=skyc1&data=peak_wind_gust&data=snowdepth&year1=2020&month1=1&day1=1&year2=2020&month2=1&day2=3&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=M&trace=T&direct=no&report_type=3&report_type=4


/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["datetime"] = df["datetime"].map(lambda x: old_timezone.localize(datetime.strptime(x, "%Y-%m-%d %H:%M")).astimezone(new_timezone))
/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cfs"] = pd.to_numeric(df['cfs'], errors='coerce')
/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:66

   station             valid  tmpf  dwpf    relh   feel  sknt  sped   alti  \
0      40B  2020-01-01 00:17  22.5  22.6  100.42      M   0.0  0.00  29.52   
1      40B  2020-01-01 01:16  22.3  23.2       M      M   2.0  2.30  29.48   
2      40B  2020-01-01 02:16  22.1  22.6       M      M   2.0  2.30  29.45   
3      40B  2020-01-01 03:16  22.1  21.9   99.16  22.10   0.0  0.00  29.42   
4      40B  2020-01-01 04:22  21.9  21.7   99.16  21.90   0.0  0.00  29.40   
5      40B  2020-01-01 05:16  21.9  21.4   97.91  21.90   0.0  0.00  29.37   
6      40B  2020-01-01 06:16  21.7  21.4   98.74  21.70   0.0  0.00  29.35   
7      40B  2020-01-01 07:16  21.6  21.6  100.00  21.60   0.0  0.00  29.35   
8      40B  2020-01-01 08:16  21.6  21.7  100.42      M   0.0  0.00  29.34   
9      40B  2020-01-01 09:16  21.6  21.9       M      M   0.0  0.00  29.33   
10     40B  2020-01-01 10:16  21.0  21.0  100.00  21.00   0.0  0.00  29.32   
11     40B  2020-01-01 11:16  18.3  18.1   99.14  18.30   0.0  0

/Users/gisaac/anaconda3/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/gisaac/anaconda3/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


LoadJob<project=hydro-earthnet-db, location=EU, id=fbe32908-8c6c-492e-acbf-7d9f39df03be>
ASOS data written to BigQuery


In [12]:
import pytz
def combine_data(h) -> None:
    tz = pytz.timezone("UTC")
    if h.asos_df.hour_updated.dt.tz is None:
        h.asos_df['hour_updated'] = h.asos_df['hour_updated'].map(lambda x: x.tz_localize("UTC"))
    joined_df = h.asos_df.merge(h.final_usgs, left_on='hour_updated', right_on='datetime', how='inner')
    nan_precip = sum(pd.isnull(joined_df['p01m']))
    nan_flow = sum(pd.isnull(joined_df['cfs']))
    h.joined_df = joined_df
    h.nan_flow = nan_flow
    h.nan_precip = nan_precip
    h.joined_df = joined_df
    h.joined_df.drop(columns=["site_no"], inplace=True)
    columns_to_drop = [col for col in h.joined_df.columns if col.endswith('_cd')]
    h.joined_df.drop(columns=columns_to_drop, inplace=True)

In [15]:
ASOS_DF

NameError: name 'ASOS_DF' is not defined

In [29]:
#combine_data(h)

In [14]:
h.joined_df

,hour_updated,p01m,valid,tmpf,dwpf,ice_accretion_1hr,mslp,drct,sped,alti,...,vsby,gust,skyc1,peak_wind_gust,snowdepth,station_id,index,datetime,cfs,height
0,2020-01-01 05:00:00+00:00,0.0,2020-01-01 04:53,40.0,33.0,NaN,1000.1,230.0,6.90,29.53,...,10.0,NaN,BKN,NaN,NaN,OWD,1,2020-01-01 05:00:00+00:00,539,3.96
1,2020-01-01 06:00:00+00:00,0.0,2020-01-01 05:53,40.0,32.0,NaN,1000.1,240.0,6.90,29.53,...,10.0,NaN,FEW,NaN,NaN,OWD,5,2020-01-01 06:00:00+00:00,533,3.95
2,2020-01-01 07:00:00+00:00,0.0,2020-01-01 06:53,39.0,28.9,NaN,1000.5,260.0,5.75,29.54,...,10.0,NaN,OVC,NaN,NaN,OWD,9,2020-01-01 07:00:00+00:00,533,3.95
3,2020-01-01 08:00:00+00:00,0.0,2020-01-01 07:53,39.0,27.0,NaN,1001.0,250.0,5.75,29.56,...,10.0,NaN,BKN,NaN,NaN,OWD,13,2020-01-01 08:00:00+00:00,533,3.95
4,2020-01-01 09:00:00+00:00,0.0,2020-01-01 08:53,38.0,26.1,NaN,1001.6,270.0,16.10,29.58,...,10.0,NaN,SCT,NaN,NaN,OWD,17,2020-01-01 09:00:00+00:00,533,3.95
5,2020-01-01 10:00:00+00:00,0.0,2020-01-01 09:53,37.0,25.0,NaN,1001.3,280.0,8.05,29.57,...,10.0,NaN,FEW,NaN,NaN,OWD,21,2020-01-01 10:00:00+00:00,533,3.95
6,2020-01-01 11:00:00+00:00,0.0,2020-01-01 10:53,36.0,25.0,NaN,1001.5,240.0,5.75,29.57,...,10.0,NaN,CLR,NaN,NaN,OWD,25,2020-01-01 11:00:00+00:00,527,3.94
7,2020-01-01 12:00:00+00:00,0.0,2020-01-01 11:53,36.0,23.0,NaN,1001.5,230.0,8.05,29.58,...,10.0,NaN,CLR,NaN,NaN,OWD,29,2020-01-01 12:00:00+00:00,533,3.95
8,2020-01-01 13:00:00+00:00,0.0,2020-01-01 12:53,36.0,23.0,NaN,1001.9,240.0,8.05,29.59,...,10.0,NaN,CLR,NaN,NaN,OWD,33,2020-01-01 13:00:00+00:00,527,3.94
9,2020-01-01 14:00:00+00:00,0.0,2020-01-01 13:53,39.0,24.1,NaN,1002.5,250.0,13.80,29.60,...,10.0,16.0,CLR,NaN,NaN,OWD,37,2020-01-01 14:00:00+00:00,533,3.95


## Add Sentinel Images

In [20]:
import pandas as pd


In [21]:
sentinel_df = pd.read_csv("/Users/gisaac/Downloads/sent_data.csv")

In [30]:
mgr = ["13TDE", "11SPB", "50RKQ"]
sentinel_df[sentinel_df["mgrs_tile"].isin(mgr)].to_csv("example1.2.csv")

In [42]:
h.final_df.drop(columns=["sensing_time"], inplace=True)

KeyError: "['sensing_time'] not found in axis"

In [24]:
sentinel_df

,mgrs_tile,base_url,sensing_time,cloud_cover
0,11SPB,gs://gcp-public-data-sentinel-2/tiles/11/S/PB/...,2017-09-01 18:39:59.64 UTC,1.0863
1,50RKQ,gs://gcp-public-data-sentinel-2/tiles/50/R/KQ/...,2017-09-02 03:04:45.62 UTC,84.0477
2,50TQS,gs://gcp-public-data-sentinel-2/tiles/50/T/QS/...,2017-09-02 03:04:45.62 UTC,1.9648
3,12SWF,gs://gcp-public-data-sentinel-2/tiles/12/S/WF/...,2017-09-02 18:03:52.86 UTC,1.4287
4,12RUP,gs://gcp-public-data-sentinel-2/tiles/12/R/UP/...,2017-09-02 18:14:13.4 UTC,92.4166
...,...,...,...,...
3092174,17RLN,gs://gcp-public-data-sentinel-2/tiles/17/R/LN/...,2019-08-17 16:15:21.005 UTC,100.0000
3092175,18FWL,gs://gcp-public-data-sentinel-2/tiles/18/F/WL/...,2019-08-27 14:56:38.222 UTC,100.0000
3092176,49SFR,gs://gcp-public-data-sentinel-2/tiles/49/S/FR/...,2019-08-06 03:19:11.486 UTC,100.0000
3092177,17TQM,gs://gcp-public-data-sentinel-2/tiles/17/T/QM/...,2019-08-17 16:10:31.174 UTC,100.0000


In [ ]:
def combine_sentinel(final_df, sentinel_df, tile):
    """ to combine the Sentinel data with the joined ASOS, USGS, and SNOTEL data.
    """
    sentinel_df = sentinel_df[sentinel_df["mgrs_tile"]==tile] 
    sentinel_df = sentinel_df[["sensing_time", "base_url"]]
    sentinel_df["sensing_time"] = pd.to_datetime(sentinel_df["sensing_time"], utc=True, format='mixed').round('60min')
    final_df =final_df.merge(sentinel_df, left_on="hour_updated", right_on="SENSING_TIME", how="left")
    return final_df

fin_df = combine_sentinel(h.final_df, sentinel_df, "11SMT")
fin_df

In [ ]:
fin_df.dropna(subset=["base_url"])

In [ ]:
 sentinel_df_11 = sentinel_df[sentinel_df["mgrs_tile"]=="11SMT"] 

In [ ]:
sentinel_df_11

In [ ]:
find = fin_df

In [ ]:
find.to_csv("final_11059300.csv")

In [ ]:
data

In [ ]:
h.final_df

In [ ]:
def write_final_df_to_bq(h, client, table_name: str) -> bool:
    sample = h.final_df[['hour_updated', 'p01m', 'valid', 'tmpf', 'dwpf', 'ice_accretion_1hr',
       'mslp', 'drct', 'sped', 'alti', 'relh', 'sknt', 'feel', 'vsby', 'gust',
       'skyc1', 'peak_wind_gust', 'snowdepth', 'cfs', 'height']]
    job = client.client.load_table_from_dataframe(sample, table_name)
    print(job.result())
    return True
h.final_df["height"] = pd.to_numeric(h.final_df["height"], errors='coerce')
h.final_df["height"] = pd.to_numeric(h.final_df["height"], errors='coerce')

In [ ]:
write_final_df_to_bq(h, h.bq_connect, "hydronet.flow7")

In [ ]:
h.joined_df.columns

In [ ]:
h.final_df["index"] = h.final_df["index"].astype('str')

In [ ]:
h.final_df["index"]

In [ ]:
write_final_df_to_bq(h, h.bq_connect, "hydronet.flow3")

In [ ]:
h.final_df["station_id"] = h.final_df["station_id"].astype('str')

In [62]:
from google.cloud import storage
client = storage.Client(project="hydro-earthnet-db")
upload_file_to_gcs(client, h.asos_df, "OWI", file_type="weather")

/Users/gisaac/anaconda3/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [58]:
from os import path
def upload_file_to_gcs(client, df, site_no, bucket_name="flow_hydro_2_data", file_type="joined_df"):
    csv_file = df.to_csv()
    bucket = client.get_bucket(bucket_name)
    gcs_path = file_type
    # Define the blob path
    blob = bucket.blob(path.join(gcs_path, site_no + ".csv"))

    # Upload the CSV data to the blob
    blob.upload_from_string(csv_file, content_type='text/csv')

In [49]:
uploa

NameError: name 'uploa' is not defined

In [61]:
h.asos_df

,hour_updated,p01m,valid,tmpf,dwpf,ice_accretion_1hr,mslp,drct,sped,alti,relh,sknt,feel,vsby,gust,skyc1,peak_wind_gust,snowdepth,station_id
0,2000-01-01 01:00:00+00:00,0.0,2000-01-01 00:53,18.0,13.0,NaN,NaN,0.0,0.00,30.14,80.29,0.0,17.96,10.0,NaN,CLR,NaN,NaN,OWD
1,2000-01-01 02:00:00+00:00,0.0,2000-01-01 01:53,17.0,13.0,NaN,NaN,0.0,0.00,30.15,83.47,0.0,17.06,10.0,NaN,CLR,NaN,NaN,OWD
2,2000-01-01 03:00:00+00:00,0.0,2000-01-01 02:53,16.0,13.0,NaN,NaN,0.0,0.00,30.16,87.47,0.0,15.98,10.0,NaN,CLR,NaN,NaN,OWD
3,2000-01-01 04:00:00+00:00,0.0,2000-01-01 03:53,15.0,13.0,NaN,NaN,0.0,0.00,30.17,90.96,0.0,15.08,9.0,NaN,CLR,NaN,NaN,OWD
4,2000-01-01 05:00:00+00:00,0.0,2000-01-01 04:53,13.0,7.0,NaN,NaN,0.0,0.00,30.18,76.68,0.0,12.92,9.0,NaN,CLR,NaN,NaN,OWD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199882,2023-01-01 20:00:00+00:00,0.0,2023-01-01 19:53,52.0,34.0,NaN,1010.3,280.0,12.65,29.84,50.12,11.0,52.00,10.0,NaN,CLR,NaN,NaN,OWD
199883,2023-01-01 21:00:00+00:00,0.0,2023-01-01 20:53,49.0,30.9,NaN,1011.6,300.0,10.35,29.87,49.63,9.0,44.53,10.0,16.0,CLR,NaN,NaN,OWD
199884,2023-01-01 22:00:00+00:00,0.0,2023-01-01 21:53,47.0,30.9,NaN,1012.2,280.0,11.50,29.89,53.50,10.0,41.65,10.0,NaN,CLR,NaN,NaN,OWD
199885,2023-01-01 23:00:00+00:00,0.0,2023-01-01 22:53,46.0,30.9,NaN,1013.1,260.0,8.05,29.92,55.36,7.0,41.84,10.0,NaN,SCT,NaN,NaN,OWD
